In [113]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

In [77]:
# Inventory Data
inventory_df = pd.read_json('./data/exported_listings.json')
stock_df = pd.read_json('./data/stock_listing_v1_1.json')
inventory_df = inventory_df.merge(stock_df, on='_id')
inventory_df = inventory_df[['date', 'title', 'category', 'unit_cost', 'current_stock', 'avg_usage_per_day']]
inventory_df.sort_values('date', inplace=True)

# ABC Model Cluster Results Data
abc_clusters_df = pd.read_csv('./data/abc_clusters.csv')

In [78]:
inventory_df

,date,title,category,unit_cost,current_stock,avg_usage_per_day
1438,2022-01-01,Infusion Pump,Equipment,224271,5,6
1439,2022-01-01,Infusion Pump,Equipment,160967,10,14
822,2022-01-01,Hospital Bed,Equipment,49936,135,1
823,2022-01-02,Hospital Bed,Equipment,36616,128,1
2611,2022-01-02,Syringe,Consumable,9,7680,662
...,...,...,...,...,...,...
2213,2024-12-30,Surgical Mask,Consumable,4,5700,1995
2803,2024-12-30,Syringe,Consumable,8,3117,289
2804,2024-12-30,Syringe,Consumable,16,5334,1767
1019,2024-12-31,Hospital Bed,Equipment,27132,64,1


In [79]:
abc_clusters_df

,title,year,month,category,unit_cost,beginning_inventory,ending_inventory,annual_usage_value,monthly_usage,annual_usage,stock_turnover_rate,stock_variability,ABC_category
0,Antiseptic Solution,2022,2022-01,Consumable,4.320000e+02,2992,2464,7.568208e+06,1459.916667,17519,0.535160,0.484987,C
1,Antiseptic Solution,2022,2022-02,Consumable,4.320000e+02,2000,2000,7.568208e+06,1459.916667,17519,0.729958,0.484987,C
2,Antiseptic Solution,2022,2022-03,Consumable,4.320000e+02,2185,526,7.568208e+06,1459.916667,17519,1.077032,0.484987,C
3,Antiseptic Solution,2022,2022-04,Consumable,4.320000e+02,1543,1198,7.568208e+06,1459.916667,17519,1.065244,0.484987,C
4,Antiseptic Solution,2022,2022-05,Consumable,4.320000e+02,1224,1213,7.568208e+06,1459.916667,17519,1.198126,0.484987,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,X-Ray-Machine,2024,2024-08,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C
643,X-Ray-Machine,2024,2024-09,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C
644,X-Ray-Machine,2024,2024-10,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C
645,X-Ray-Machine,2024,2024-11,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C


<h2 style="padding: 0.5rem; background-color: #513d5c; color: white;">Feature Engineering</h2>

In [81]:
inventory_df['date'] = pd.to_datetime(inventory_df['date']) # Preprocessing: Datetime Conversion
inventory_df['month'] = inventory_df['date'].dt.to_period('M')
inventory_df = inventory_df[['date', 'month', 'title', 'category', 'unit_cost', 'current_stock', 'avg_usage_per_day']]
inventory_df

,date,month,title,category,unit_cost,current_stock,avg_usage_per_day
1438,2022-01-01,2022-01,Infusion Pump,Equipment,224271,5,6
1439,2022-01-01,2022-01,Infusion Pump,Equipment,160967,10,14
822,2022-01-01,2022-01,Hospital Bed,Equipment,49936,135,1
823,2022-01-02,2022-01,Hospital Bed,Equipment,36616,128,1
2611,2022-01-02,2022-01,Syringe,Consumable,9,7680,662
...,...,...,...,...,...,...,...
2213,2024-12-30,2024-12,Surgical Mask,Consumable,4,5700,1995
2803,2024-12-30,2024-12,Syringe,Consumable,8,3117,289
2804,2024-12-30,2024-12,Syringe,Consumable,16,5334,1767
1019,2024-12-31,2024-12,Hospital Bed,Equipment,27132,64,1


In [82]:
inventory_df = inventory_df.copy().sort_values('date')
inventory_df['stock_diff'] = inventory_df.groupby(['title', 'month'])[['current_stock']].transform(lambda x: x.diff())
inventory_df['restock_quantity'] = inventory_df.groupby(['title', 'month'])['stock_diff'].transform(lambda x: x.clip(lower=0))
inventory_df

,date,month,title,category,unit_cost,current_stock,avg_usage_per_day,stock_diff,restock_quantity
1438,2022-01-01,2022-01,Infusion Pump,Equipment,224271,5,6,NaN,NaN
1439,2022-01-01,2022-01,Infusion Pump,Equipment,160967,10,14,5.0,5.0
822,2022-01-01,2022-01,Hospital Bed,Equipment,49936,135,1,NaN,NaN
823,2022-01-02,2022-01,Hospital Bed,Equipment,36616,128,1,-7.0,0.0
2611,2022-01-02,2022-01,Syringe,Consumable,9,7680,662,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2803,2024-12-30,2024-12,Syringe,Consumable,8,3117,289,-2011.0,0.0
2213,2024-12-30,2024-12,Surgical Mask,Consumable,4,5700,1995,-3495.0,0.0
2804,2024-12-30,2024-12,Syringe,Consumable,16,5334,1767,2217.0,2217.0
1019,2024-12-31,2024-12,Hospital Bed,Equipment,27132,64,1,37.0,37.0


In [83]:
# Checker
inventory_df[inventory_df['title'] == 'Infusion Pump']
inventory_df[(inventory_df['title'] == 'ECG Machine') & (inventory_df['month'] == '2022-05')]

,date,month,title,category,unit_cost,current_stock,avg_usage_per_day,stock_diff,restock_quantity
1044,2022-05-03,2022-05,ECG Machine,Equipment,256447,9,2,NaN,NaN
1045,2022-05-20,2022-05,ECG Machine,Equipment,362581,9,0,0.0,0.0


In [84]:
restock_df = inventory_df.groupby(['title', 'month'])[['restock_quantity']].sum().reset_index()
restock_df

,title,month,restock_quantity
0,Antiseptic Solution,2022-01,0.0
1,Antiseptic Solution,2022-02,0.0
2,Antiseptic Solution,2022-03,3014.0
3,Antiseptic Solution,2022-04,0.0
4,Antiseptic Solution,2022-05,4754.0
...,...,...,...
641,X-Ray-Machine,2024-08,10.0
642,X-Ray-Machine,2024-09,5.0
643,X-Ray-Machine,2024-10,21.0
644,X-Ray-Machine,2024-11,9.0


In [85]:
# Create a MONTHLY BASED BEGINNING/ENDING INVENTORY FOR EQUIPMENT (It was annually-based on imported data)
inventory_df['date'] = pd.to_datetime(inventory_df['date'])
inventory_df['month'] = inventory_df['date'].dt.to_period('M')
inventory_df['year'] = inventory_df['date'].dt.to_period('Y')

monthly_start_end_stock = inventory_df.groupby(['title', 'month']).agg(
        beginning_inventory_monthly = ('current_stock', 'first'),
        ending_inventory_monthly = ('current_stock', 'last'),
).reset_index()

restock_df = restock_df.merge(monthly_start_end_stock, on=['title', 'month'])
restock_df

,title,month,restock_quantity,beginning_inventory_monthly,ending_inventory_monthly
0,Antiseptic Solution,2022-01,0.0,2992,2464
1,Antiseptic Solution,2022-02,0.0,2000,2000
2,Antiseptic Solution,2022-03,3014.0,2185,526
3,Antiseptic Solution,2022-04,0.0,1543,1198
4,Antiseptic Solution,2022-05,4754.0,1224,1213
...,...,...,...,...,...
641,X-Ray-Machine,2024-08,10.0,6,6
642,X-Ray-Machine,2024-09,5.0,2,3
643,X-Ray-Machine,2024-10,21.0,5,5
644,X-Ray-Machine,2024-11,9.0,1,2


In [86]:
# Preparation for Merge
restock_df['month'] = abc_clusters_df['month'].astype(str)

# Merge to ABC Clusters (to merge for beginning/ending inventory)
restock_df1 = restock_df.merge(abc_clusters_df, on=['title', 'month'])
restock_df1

,title,month,restock_quantity,beginning_inventory_monthly,ending_inventory_monthly,year,category,unit_cost,beginning_inventory,ending_inventory,annual_usage_value,monthly_usage,annual_usage,stock_turnover_rate,stock_variability,ABC_category
0,Antiseptic Solution,2022-01,0.0,2992,2464,2022,Consumable,4.320000e+02,2992,2464,7.568208e+06,1459.916667,17519,0.535160,0.484987,C
1,Antiseptic Solution,2022-02,0.0,2000,2000,2022,Consumable,4.320000e+02,2000,2000,7.568208e+06,1459.916667,17519,0.729958,0.484987,C
2,Antiseptic Solution,2022-03,3014.0,2185,526,2022,Consumable,4.320000e+02,2185,526,7.568208e+06,1459.916667,17519,1.077032,0.484987,C
3,Antiseptic Solution,2022-04,0.0,1543,1198,2022,Consumable,4.320000e+02,1543,1198,7.568208e+06,1459.916667,17519,1.065244,0.484987,C
4,Antiseptic Solution,2022-05,4754.0,1224,1213,2022,Consumable,4.320000e+02,1224,1213,7.568208e+06,1459.916667,17519,1.198126,0.484987,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,X-Ray-Machine,2024-07,10.0,6,6,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C
642,X-Ray-Machine,2024-08,5.0,2,3,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C
643,X-Ray-Machine,2024-09,21.0,5,5,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C
644,X-Ray-Machine,2024-10,9.0,1,2,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C


In [87]:
restock_df1['demand'] = restock_df1['beginning_inventory_monthly'] - restock_df1['ending_inventory_monthly'] + restock_df1['restock_quantity']
restock_df1[['title', 'month', 'demand']]

,title,month,demand
0,Antiseptic Solution,2022-01,528.0
1,Antiseptic Solution,2022-02,0.0
2,Antiseptic Solution,2022-03,4673.0
3,Antiseptic Solution,2022-04,345.0
4,Antiseptic Solution,2022-05,4765.0
...,...,...,...
641,X-Ray-Machine,2024-07,10.0
642,X-Ray-Machine,2024-08,4.0
643,X-Ray-Machine,2024-09,21.0
644,X-Ray-Machine,2024-10,8.0


<div style="width: 30%;">
    <h3 style="background-color: #a07fb3; color: white; padding: 0.5rem 0.5rem; text-align: center; vertical-align: middle; box-shadow: 5px 5px #4d3e66;">🎯 Feature Selection</h3>
    <div style="margin: 1.5rem 0;"></div>
</div>

In [89]:
# Augmented Dickey-Fuller Test (ADF) --> stationarity check for the ARIMA
def adf_test(series):
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print('✅ Data is stationary.')
        return True
    else:
        print('❌ Data is NOT stationary. Differencing is needed.')
        return False

In [90]:
restock_df1.dtypes

title                           object
month                           object
restock_quantity               float64
beginning_inventory_monthly      int64
ending_inventory_monthly         int64
year                             int64
category                        object
unit_cost                      float64
beginning_inventory              int64
ending_inventory                 int64
annual_usage_value             float64
monthly_usage                  float64
annual_usage                     int64
stock_turnover_rate            float64
stock_variability              float64
ABC_category                    object
demand                         float64
dtype: object

In [91]:
restock_df1['month'] = pd.to_datetime(restock_df1['month']).dt.to_period('M')
restock_df1.asfreq('MS')
restock_df1

,title,month,restock_quantity,beginning_inventory_monthly,ending_inventory_monthly,year,category,unit_cost,beginning_inventory,ending_inventory,annual_usage_value,monthly_usage,annual_usage,stock_turnover_rate,stock_variability,ABC_category,demand
0,Antiseptic Solution,2022-01,0.0,2992,2464,2022,Consumable,4.320000e+02,2992,2464,7.568208e+06,1459.916667,17519,0.535160,0.484987,C,528.0
1,Antiseptic Solution,2022-02,0.0,2000,2000,2022,Consumable,4.320000e+02,2000,2000,7.568208e+06,1459.916667,17519,0.729958,0.484987,C,0.0
2,Antiseptic Solution,2022-03,3014.0,2185,526,2022,Consumable,4.320000e+02,2185,526,7.568208e+06,1459.916667,17519,1.077032,0.484987,C,4673.0
3,Antiseptic Solution,2022-04,0.0,1543,1198,2022,Consumable,4.320000e+02,1543,1198,7.568208e+06,1459.916667,17519,1.065244,0.484987,C,345.0
4,Antiseptic Solution,2022-05,4754.0,1224,1213,2022,Consumable,4.320000e+02,1224,1213,7.568208e+06,1459.916667,17519,1.198126,0.484987,C,4765.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,X-Ray-Machine,2024-07,10.0,6,6,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C,10.0
642,X-Ray-Machine,2024-08,5.0,2,3,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C,4.0
643,X-Ray-Machine,2024-09,21.0,5,5,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C,21.0
644,X-Ray-Machine,2024-10,9.0,1,2,2024,Equipment,3.757779e+06,4,8,1.127334e+08,2.500000,30,5.000000,0.448397,C,8.0


In [133]:
#X_features = ['title', 'month', 'demand']
X_features = ['title', 'month', 'demand', 'restock_quantity', 'ABC_category']
X_all = restock_df1[X_features]

# One-hot encode the ABC_category
X_all = pd.get_dummies(X_all, columns=['ABC_category'], prefix='ABC')
X_all[['ABC_A', 'ABC_B', 'ABC_C']] = X_all[['ABC_A', 'ABC_B', 'ABC_C']].astype(int)

for item in X_all['title'].unique():
    
    # Filter data for the specific item
    X = X_all[X_all['title'] == item].copy()
    X = X.set_index('month')
    X = X.drop('title', axis=1)

    # DATA STATIONARITY CHECK
    is_stationary = adf_test(X['demand'])
    
    if not is_stationary:
        X['demand_diff'] = X['demand'].diff()
        X.dropna(inplace=True)
        stepwise_fit = auto_arima(X['demand_diff'], seasonal=False, stepwise=True, suppress_warnings=True, trace=True)
        p, d, q = stepwise_fit.order
        model = SARIMAX(
            X['demand_diff'], 
            exog=X[['restock_quantity', 'ABC_A', 'ABC_B', 'ABC_C']], 
            order=(p, d, q)
        )

    else:
        stepwise_fit = auto_arima(X['demand'], seasonal=False, stepwise=True, suppress_warnings=True, trace=True)
        p, d, q = stepwise_fit.order
        model = SARIMAX(
            X['demand'], 
            exog=X[['restock_quantity', 'ABC_A', 'ABC_B', 'ABC_C']], 
            order=(p, d, q))
    
    # MODEL FITTING
    model_fit = model.fit()
    print(model_fit.summary())

    # DEMAND FORECASTING
    future_steps = 30

    # Generate future exogenous values
    future_exog = pd.DataFrame({
        'restock_quantity': [X['restock_quantity'].mean()] * future_steps,  # Use mean or estimated values
        'ABC_A': [0] * future_steps,  # Adjust based on expected future ABC categories
        'ABC_B': [1] * future_steps,  # Adjust as needed
        'ABC_C': [0] * future_steps   # Adjust as needed
    })

    # Forecasting with exogenous variables
    forecast = model_fit.forecast(steps=future_steps, exog=future_exog)
    
    print(f'ITEM NAME: {item}')
    print(forecast)

ADF Statistic: -8.153846600790363
p-value: 9.536330827483064e-13
✅ Data is stationary.
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=618.526, Time=0.11 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=649.083, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=637.521, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=644.813, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=inf, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=inf, Time=0.06 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=inf, Time=0.12 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=0.11 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=0.03 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=inf, Time=0.09 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=inf, Time=0.08 sec
 ARIMA(3,0,3)(0,0,0)[0]             : AIC=inf, Time=0.14 sec
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec

Best model:  ARIMA(2,0,2)(0,0,0)[0]          
Total fit time

C:\Users\Lance Ventura\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                 demand   No. Observations:                   35
Model:               SARIMAX(2, 0, 2)   Log Likelihood                -278.568
Date:                Thu, 20 Feb 2025   AIC                            575.136
Time:                        23:32:22   BIC                            589.134
Sample:                    01-31-2022   HQIC                           579.968
                         - 12-31-2024                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
restock_quantity     0.8960      0.055     16.334      0.000       0.788       1.003
ABC_A                     0      0.000          0      1.000      -0.001       0.001
ABC_B                     0 

C:\Users\Lance Ventura\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\mlemodel.py:3016: RuntimeWarning: invalid value encountered in divide
  return self.params / self.bse


                               SARIMAX Results                                
Dep. Variable:                 demand   No. Observations:                   36
Model:                        SARIMAX   Log Likelihood                -318.441
Date:                Thu, 20 Feb 2025   AIC                            646.883
Time:                        23:32:24   BIC                            654.800
Sample:                    01-31-2022   HQIC                           649.646
                         - 12-31-2024                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
restock_quantity     0.5659      0.186      3.036      0.002       0.201       0.931
ABC_A                     0   1.06e-08          0      1.000   -2.07e-08    2.07e-08
ABC_B                     0 

...

                               SARIMAX Results                                
Dep. Variable:                 demand   No. Observations:                   36
Model:                        SARIMAX   Log Likelihood                 -29.872
Date:                Thu, 20 Feb 2025   AIC                             69.743
Time:                        23:32:30   BIC                             77.661
Sample:                    01-31-2022   HQIC                            72.507
                         - 12-31-2024                                         
Covariance Type:                  opg                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
restock_quantity     0.5478      0.152      3.611      0.000       0.250       0.845
ABC_A                0.6413      0.180      3.564      0.000       0.289       0.994
ABC_B                     0 

C:\Users\Lance Ventura\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\mlemodel.py:3016: RuntimeWarning: invalid value encountered in divide
  return self.params / self.bse


 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.10 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=322.906, Time=0.00 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=283.434, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=305.708, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=281.028, Time=0.02 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=280.359, Time=0.02 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=278.298, Time=0.03 sec
 ARIMA(5,0,0)(0,0,0)[0]             : AIC=280.035, Time=0.04 sec
 ARIMA(4,0,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec
 ARIMA(5,0,1)(0,0,0)[0]             : AIC=inf, Time=0.13 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept   : AIC=266.869, Time=0.08 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept   : AIC=264.869, Time=0.04 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=263.806, Time=0.06 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=262.729, Time=0.02 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC

C:\Users\Lance Ventura\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\mlemodel.py:3016: RuntimeWarning: invalid value encountered in divide
  return self.params / self.bse


 ARIMA(2,0,0)(0,0,0)[0]             : AIC=285.927, Time=0.02 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=285.566, Time=0.02 sec
 ARIMA(4,0,0)(0,0,0)[0]             : AIC=286.984, Time=0.03 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=287.108, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=285.328, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=285.282, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=282.731, Time=0.05 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=301.620, Time=0.02 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=inf, Time=0.09 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=300.947, Time=0.03 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=0.11 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.10 sec

Best model:  ARIMA(1,0,2)(0,0,0)[0]          
Total fit time: 0.659 seconds
                               SARIMAX Results                                
Dep. Variable:                 demand   No. Observations:                   